In [1]:
from pyspark.sql import SparkSession
import csv
import pandas as pd

spark = SparkSession.builder.master("local").appName("lopputyo").getOrCreate()

df = spark.read.csv("merged19_20_21.csv", header=True, inferSchema=True)
df.show(vertical=True)

-RECORD 0----------------------------------
 aikatyyppi         | 0                    
 kuukausi_nro       | Vuosikertymä         
 vuosikuukausi      | 12                   
 vuosi              | 201912               
 kunta_nro          | 2019                 
 kunta_nimi         | 20                   
 ikaryhma           | Akaa                 
 sukupuoli          | 25-29                
 etuus              | Mies                 
 korvausperuste     | Työmarkkinatukiet... 
 saaja_lkm          | Työllistymistä ed... 
 korvattu_paiva_lkm | 19                   
 maksettu_eur       | 3158                 
-RECORD 1----------------------------------
 aikatyyppi         | 1                    
 kuukausi_nro       | Vuosikertymä         
 vuosikuukausi      | 12                   
 vuosi              | 201912               
 kunta_nro          | 2019                 
 kunta_nimi         | 19                   
 ikaryhma           | Aura                 
 sukupuoli          | 25-29     

In [2]:
df.printSchema()

root
 |-- aikatyyppi: integer (nullable = true)
 |-- kuukausi_nro: string (nullable = true)
 |-- vuosikuukausi: integer (nullable = true)
 |-- vuosi: integer (nullable = true)
 |-- kunta_nro: integer (nullable = true)
 |-- kunta_nimi: integer (nullable = true)
 |-- ikaryhma: string (nullable = true)
 |-- sukupuoli: string (nullable = true)
 |-- etuus: string (nullable = true)
 |-- korvausperuste: string (nullable = true)
 |-- saaja_lkm: string (nullable = true)
 |-- korvattu_paiva_lkm: integer (nullable = true)
 |-- maksettu_eur: integer (nullable = true)



In [7]:
df.registerTempTable("example")
df.groupBy('etuus').count().orderBy('count',ascending=False).show()

+----------+-------+
|     etuus|  count|
+----------+-------+
|      Mies|2162114|
|    Nainen|2131714|
|Tuntematon| 490511|
+----------+-------+

